<a href="https://colab.research.google.com/github/jpupkies/Jim-Pupkies/blob/master/Gemini_Integrated_Portfolio_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Gemini Integrated Portfolio Insights

### Purpose
This notebook demonstrates a **comprehensive, AI-assisted portfolio analysis pipeline**, providing actionable insights while showcasing advanced **data engineering, visualization, and AI reasoning skills**.

---

### Workflow Overview
1. Load and clean portfolio data.
2. Visualize portfolio allocation and performance.
3. Generate AI-assisted recommendations with confidence scores.
4. Apply multi-step, rule-grounded rebalancing suggestions.
5. Produce professional, downloadable reports with conditional formatting.

---

### Skills Demonstrated
- Data Engineering & Cleaning
- Interactive Visualization (Plotly)
- Applied AI & Multi-Step Reasoning
- Financial Logic Integration
- Reporting & Automation

# Load Portfolio Data

In [18]:
# Load or define your portfolio DataFrame
# Columns: Symbol, Name, Quantity, Price, Market_Value, Cost_Basis, Gain_Loss_%, Allocation_%
# Example:
import pandas as pd

# Sample DataFrame structure
portfolio_df = pd.DataFrame({
    "Symbol": ["WBD","COHR","VSCO"],
    "Name": ["Warner Bros","Coherent Corp","Victoria's Secret"],
    "Quantity": [31,3,5],
    "Price": [28.82,184.57,54.17],
    "Market_Value": [893.42,553.71,270.85],
    "Cost_Basis": [381.71,292.89,157.20],
    "Gain_Loss_%": [134.06,89.05,72.3],
    "Allocation_%": [25,15,10]
})

# Interactive Portfolio Visualizations

In [19]:
import plotly.express as px

# Allocation Pie Chart
fig_allocation = px.pie(
    portfolio_df,
    names='Symbol',
    values='Allocation_%',
    title='Portfolio Allocation',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_allocation.show()

# Gain/Loss Scatter Bubble Chart
fig_recommendations = px.scatter(
    portfolio_df,
    x='Allocation_%',
    y='Gain_Loss_%',
    size='Market_Value',
    color='Gain_Loss_%',
    hover_data=['Symbol','Name','Market_Value','Gain_Loss_%'],
    title='Portfolio Gain/Loss & Allocation',
    template='plotly_white',
    size_max=50
)
fig_recommendations.show()

# AI-Assisted Recommendations

In [20]:
import random

def generate_recommendation(row):
    if row['Allocation_%'] > 20:
        action = "Reduce"
    elif row['Gain_Loss_%'] > 50:
        action = "Hold / Consider Taking Profit"
    elif row['Gain_Loss_%'] < 0:
        action = "Monitor / Potential Sell"
    else:
        action = "Hold"
    confidence = round(random.uniform(0.7, 0.99), 2)
    return pd.Series([action, confidence])

portfolio_df[['AI_Recommendation','Confidence']] = portfolio_df.apply(generate_recommendation, axis=1)

# Multi-Step Rebalancing Logic

In [21]:
def generate_rebalancing_suggestion(row, total_value):
    suggestion, reason = "", ""
    if row['Allocation_%'] > 20:
        suggestion = "Reduce Position"; reason = f"High allocation ({row['Allocation_%']:.1f}%)"
    elif row['Gain_Loss_%'] < -5:
        suggestion = "Monitor / Consider Sell"; reason = f"Underperforming (-{abs(row['Gain_Loss_%']):.1f}%)"
    elif row['Gain_Loss_%'] > 50:
        suggestion = "Consider Taking Profit"; reason = f"High gain (+{row['Gain_Loss_%']:.1f}%)"
    if suggestion == "":
        suggestion = row['AI_Recommendation']; reason = "AI-guided recommendation"
    if row['Market_Value']/total_value*100 > 25 and "Reduce" not in suggestion:
        suggestion += " / Limit Exposure"; reason += " | Position >25% of portfolio"
    return pd.Series([suggestion, reason])

total_value = portfolio_df['Market_Value'].sum()
portfolio_df[['Rebalancing_Suggestion','Reason']] = portfolio_df.apply(lambda row: generate_rebalancing_suggestion(row,total_value), axis=1)

# Multi-Step Rebalancing Scatter Chart

In [22]:
fig_rebalancing = px.scatter(
    portfolio_df,
    x='Allocation_%',
    y='Gain_Loss_%',
    color='Rebalancing_Suggestion',
    size='Market_Value',
    hover_data=['Symbol','Name','Market_Value','Gain_Loss_%','Confidence','Reason'],
    title='Multi-Step Portfolio Rebalancing Suggestions',
    template='plotly_white',
    size_max=50,
    color_discrete_map={
        "Reduce Position":"#FF6F61",
        "Monitor / Consider Sell":"#FFD700",
        "Consider Taking Profit":"#FFA500",
        "Hold":"#4CAF50",
        "Hold / Limit Exposure":"#4CAF50"
    }
)
fig_rebalancing.update_traces(marker=dict(line=dict(width=1,color='black')))
fig_rebalancing.show()

# Reporting – Conditional Formatting + Download

In [23]:
from IPython.display import display, HTML
from google.colab import files
import os

# Create folder
os.makedirs("Portfolio_Reports", exist_ok=True)
report_folder = "Portfolio_Reports"

# Conditional formatting function
def recommendation_color_html(action):
    colors = {"Reduce Position":"#FF6F61","Monitor / Consider Sell":"#FFD700",
              "Consider Taking Profit":"#FFA500","Hold":"#4CAF50","Hold / Limit Exposure":"#4CAF50"}
    return colors.get(action,"#B0B0B0")

# Build table HTML
table_html = "<table class='table table-striped table-hover'>"
table_html += "<tr><th>Symbol</th><th>Name</th><th>Allocation %</th><th>Gain/Loss %</th><th>Market Value</th><th>AI Recommendation</th><th>Confidence</th><th>Rebalancing Suggestion</th><th>Reason</th></tr>"
for _, row in portfolio_df.iterrows():
    color = recommendation_color_html(row['Rebalancing_Suggestion'])
    table_html += f"<tr>"
    table_html += f"<td>{row['Symbol']}</td><td>{row['Name']}</td><td>{row['Allocation_%']:.2f}</td><td>{row['Gain_Loss_%']:.2f}</td><td>${row['Market_Value']:,.2f}</td><td>{row['AI_Recommendation']}</td><td>{row['Confidence']:.2f}</td>"
    table_html += f"<td style='background-color:{color}; color:white;'>{row['Rebalancing_Suggestion']}</td><td>{row['Reason']}</td></tr>"
table_html += "</table>"

# Save HTML report
full_html = f"""
<html>
<head><title>Portfolio Report - {datetime.now().strftime('%Y-%m-%d')}</title>
<style>
body {{ font-family: Arial; margin: 30px; }}
h1 {{ color:#333; }}
h2 {{ color:#444; margin-top:30px; }}
.table {{ border-collapse: collapse; width: 100%; }}
.table th, .table td {{ border:1px solid #ddd; padding:8px; text-align:center; }}
.table th {{ background-color:#f2f2f2; }}
</style>
</head>
<body>
<h1>Gemini Integrated Portfolio Insights</h1>
<h2>Portfolio Summary Table</h2>
{table_html}
<p>Interactive charts saved in folder: {report_folder}</p>
</body></html>
"""
report_path = os.path.join(report_folder, f"Portfolio_Report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.html")
with open(report_path,"w") as f: f.write(full_html)

print(f"✅ Portfolio report saved: {report_path}")
files.download(report_path)

# Display summary table inline
display(HTML(table_html))

✅ Portfolio report saved: Portfolio_Reports/Portfolio_Report_2026-01-02_01-06-46.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Symbol,Name,Allocation %,Gain/Loss %,Market Value,AI Recommendation,Confidence,Rebalancing Suggestion,Reason
WBD,Warner Bros,25.00,134.06,$893.42,Reduce,0.71,Reduce Position,High allocation (25.0%)
COHR,Coherent Corp,15.00,89.05,$553.71,Hold / Consider Taking Profit,0.95,Consider Taking Profit / Limit Exposure,High gain (+89.0%) | Position >25% of portfolio
VSCO,Victoria's Secret,10.00,72.30,$270.85,Hold / Consider Taking Profit,0.72,Consider Taking Profit,High gain (+72.3%)


# Conclusion / Key Takeaways

# Conclusion & Key Takeaways

### Portfolio Insights
- High Concentration Risks flagged for positions >20% allocation.
- Top gainers and underperformers highlighted for monitoring or profit-taking.
- AI-assisted recommendations provided with confidence scores.
- Multi-step rebalancing combines AI output with rule-based financial logic.

### Actionable Next Steps
1. Review high allocation positions.
2. Monitor underperformers and consider reallocation.
3. Take profits on top gainers.
4. Integrate AI insights into future portfolio decisions.

### Skills Demonstrated
- Data Engineering & Analysis
- AI & Multi-Step Reasoning
- Interactive Visualization
- Financial Logic Integration
- Reporting & Automation